In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth
# c_path = "/content/outputs/checkpoint-282"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.12.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from transformers import TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
data_files = {"train": "/content/train.csv", "validation": "/content/val.csv"}
dataset = load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# classification_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# Carefully read the following post and classify it as 'Yes' (contains signs of suicide risk) or 'No' (does not contain signs of suicide risk) only.

# ### Input:
# {}

# ### Response:
# {}
# """

In [ ]:
classification_prompt = """
**Context**:
- You are a highly experienced psychology expert specializing in suicide assessment.

**Instructions**:
- Carefully read the given post (somewhere on the Internet).
- Base your decision on the explicit content of the post. Avoid assumptions beyond what is stated.
- Your task is to perform a detailed step-by-step analysis to determine whether the given post shows signs of stress, then classify the post into 'Yes' or 'No' suicide.
- Follow a step-by-step reasoning process using Chain-of-Thought (CoT) to determine the outcome. Do not write out the intermediate reasoning steps, only the final conclusion. Follow these steps:
  1. Identify the primary emotion.
  2. Detect sensitive keywords related to suicide.
  3. Assess the severity level.
  4. Identify any signs of harmful or dangerous thoughts.
  5. Summarize the findings.

### Example 1:
**Post**: "I've been juggling too many responsibilities lately, and I can't seem to focus on anything properly."
**Reasoning**:
1. The post mentions "juggling too many responsibilities," indicating excessive workload.
2. The phrase "I can't seem to focus" suggests mental exhaustion, a common symptom of stress.
3. Together, these factors indicate the presence of stress.
**Final Answer**: Yes

### Example 2:
**Post**: "I feel so relaxed after my vacation. Everything seems manageable now."
**Reasoning**:
1. The post describes positive experiences ("relaxed after my vacation").
2. The phrase "everything seems manageable" indicates the absence of stress-related difficulties.
3. There are no mentions of overwhelming tasks, anxiety, or mental strain.
**Final Answer**: No

### Example 3:
**Post**: "I haven't been sleeping well, and my work deadlines are piling up. It's starting to feel like too much to handle."
**Reasoning**:
1. The post mentions "haven't been sleeping well," a symptom often linked to stress.
2. "Work deadlines are piling up" indicates external pressures contributing to mental strain.
3. "It's starting to feel like too much to handle" directly reflects the person's sense of being overwhelmed.
**Final Answer**: Yes

**Post for Classification**:
"{}"

**Output Format**:
- Do not provide any additional explanation or commentary or punctuation. Only output is Yes or No.

**Respone**:
"{}"
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
def format_data(examples):
    posts = examples["post"]
    labels = examples["is_suicide"]
    texts = [
        classification_prompt.format(post, label) + EOS_TOKEN
        for post, label in zip(posts, labels)
    ]
    return {"text": texts}

In [ ]:
dataset = dataset.map(format_data, batched=True)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [ ]:
def convert_labels(examples):
    examples["labels"] = [1 if label == "Yes" else 0 for label in examples["is_suicide"]]
    return examples

In [ ]:
dataset = dataset.map(convert_labels, batched=True)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][0])

{'post': 'hi really know phrase situation try life really good point right never really depressed stuff 99 percent time mind clear graduate high school really excited however people family friend group ton issue wether sleep day hate ambition keep living world got problem wanna sound like gloating usually person lot people end going usually able talk people issue help long run yeah sometimes issue make really sad stuff feel sad people telling feel worthless today one best friend showed wa cutting really effected talked mostly painfully bored even know pretty logical guy go class sit hour time nothing challenging way school work ha clinically diagnosed depression ha psychiatric ward see therapist regularly couple hour since showed arm even see got two main problem right first really know help said literally thing really hang school weekend sit bed day trying make effort much need said long go highschool considering law going feel way graduate need know something help tell parent therapi

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        output_dir="outputs",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        fp16=True,
        logging_steps=10,
        optim = "adamw_8bit",
        save_strategy="epoch",
        save_total_limit=2,
        weight_decay=0.01,
        seed=42,
    ),
)

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,516 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 282
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.877100
20,1.195000
30,1.168000
40,1.112100
50,1.087100
60,1.064700
70,1.135700
80,1.263300
90,1.083300
100,0.869200


TrainOutput(global_step=282, training_loss=1.0366793803289427, metrics={'train_runtime': 6294.3095, 'train_samples_per_second': 0.723, 'train_steps_per_second': 0.045, 'total_flos': 1.0706704998049382e+17, 'train_loss': 1.0366793803289427, 'epoch': 2.970976253298153})

### Evaluate

In [ ]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
inputs = tokenizer(
[
    classification_prompt.format(
        "shit like get better everyone purpose need find get better wait one day glad kill self course suicide permanent solution blah blah blah fuck shit even better people religion say pray suicidal people tired hearing generic shit open",
        "",
    )
], return_tensors = "pt").to("cuda")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [ ]:
batch_size = 4
valid_texts = dataset["validation"]["post"]
valid_labels = dataset["validation"]["is_suicide"]
valid_predictions = []

In [ ]:
for i in tqdm(range(0, len(valid_texts), batch_size), desc="Processing Validation Set"):
    batch_posts = valid_texts[i : i + batch_size]

    batch_inputs = [
        classification_prompt.format(post, "") for post in batch_posts
    ]

    inputs = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    clean_predictions = [
        output.strip().split("\n")[-1]
        for output in decoded_outputs
    ]
    valid_predictions.extend(clean_predictions)

Processing Validation Set: 100%|██████████| 95/95 [06:02<00:00,  3.82s/it]


In [ ]:
print("Sample Predictions:")
for i, pred in enumerate(valid_predictions[:5]):
    print(f"Post {i + 1}: {valid_texts[i]}")
    print(f"Prediction: {pred}")
    print(f"True Label: {valid_labels[i]}")
    print("---")

Sample Predictions:
Post 1: indubitably worth anything contrary belief redditors reply never done anything correct life ever succeeded ha hurtful disrespectful others taking others granted breed catastrophe talented skilled anything particular average every concept known man motivation intention progress life seem inconvenience unpleasant burden towards society say existence comparable tumor productive produce negative effect others unfortunately unbearable pain come interaction someone like entrapped body dream going eternal sleep great rest last end time interest anything anymore daily politics bore great deal intrigued life see reason follow intrinsic value drive intent live
Prediction: ""
True Label: Yes
---
Post 2: almost everyday wake hating would hate though graduated high school loving healthy family hobby online friend yet find paranoid hateful deserve life sometimes wanna scream feeling like trapped another person skin feel like accomplishment fake useless slab meat eventuall

In [ ]:
binary_predictions = [1 if pred == "Yes" else 0 for pred in valid_predictions]
binary_labels = [1 if label == "Yes" else 0 for label in valid_labels]

accuracy = accuracy_score(binary_labels, binary_predictions)
report = classification_report(binary_labels, binary_predictions, target_names=["No", "Yes"])

print(report)

              precision    recall  f1-score   support

          No       0.49      1.00      0.66       186
         Yes       0.00      0.00      0.00       193

    accuracy                           0.49       379
   macro avg       0.25      0.50      0.33       379
weighted avg       0.24      0.49      0.32       379



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
